In [1]:
import clickhouse_connect
import pandas as pd
import duckdb
import plotly.express as px
import plotly.io as pio
import base64
import ollama
from io import BytesIO

## obtain data to parquet file

In [60]:
client = clickhouse_connect.get_client(
    host='43.216.85.155',
    port='8123'
)

mmsi = 533045400

qry =  f'''
    SELECT ts, mmsi, messageType, sog, cog, rot, trueHeading, latitude, longitude
    FROM pnav.ais_position
    WHERE mmsi={mmsi} AND ts >= '2025-05-28 00:00:00' AND ts < '2025-06-05 00:00:00' AND longitude >= 0 AND longitude <=180
    ORDER BY ts
'''

result = client.query(qry)

# convert clickhouse result to dataframe
dbdf = pd.DataFrame(result.result_rows)
dbdf.columns = list(result.column_names)

In [61]:
dbdf.to_parquet(f'./data/{mmsi}.parquet', index=False)

In [59]:
duckdb.sql(f'''
    SELECT *
    FROM './data/{mmsi}.parquet'
    ORDER BY ts
''').fetchdf()

,ts,mmsi,messageType,sog,cog,rot,trueHeading,latitude,longitude
0,2025-05-28 01:10:59.420,312272000,3,0.2,60.6,-731.386484,511,1.851777,104.740217
1,2025-05-28 01:11:18.250,312272000,1,0.1,98.2,-731.386484,511,1.851767,104.740250
2,2025-05-28 01:11:28.657,312272000,1,0.1,133.2,-731.386484,511,1.851718,104.740253
3,2025-05-28 01:11:49.704,312272000,1,0.2,179.4,-731.386484,511,1.851645,104.740293
4,2025-05-28 01:12:18.861,312272000,1,0.2,88.0,-731.386484,511,1.851738,104.740247
...,...,...,...,...,...,...,...,...,...
19010,2025-06-03 18:28:31.979,312272000,1,3.9,248.7,-0.000000,245,1.185290,103.825860
19011,2025-06-03 18:28:51.211,312272000,1,3.8,250.0,-0.000000,244,1.185080,103.825410
19012,2025-06-03 18:29:04.484,312272000,1,3.6,247.8,-0.000000,244,1.185005,103.825243
19013,2025-06-04 23:11:23.448,312272000,3,102.3,360.0,-0.000000,187,1.230922,103.585487


In [5]:
from polygons import *
outter_restricted_region = get_outter_restricted_region_setting()

In [6]:
outter_restricted_region

{'type': 'Polygon',
 'coordinates': [[[104.26306444444444, 2.583864444444444],
   [105.25033277777777, 2.583864444444444],
   [105.25033277777777, 1.3895897222222222],
   [104.26306444444444, 1.3895897222222222],
   [104.26306444444444, 2.583864444444444]]]}

In [2]:
client = clickhouse_connect.get_client(
    host='43.216.85.155',
    port='8123'
)

qry = f'''
    select *
    from pnav.ptp_meteorological
    where ts >= '2025-10-15' and ts < '2025-11-01'
'''


result = client.query(qry)

In [3]:
df = pd.DataFrame(result.result_rows)
df.columns = list(result.column_names)

df['ts'] = pd.to_datetime(df['ts'])  

In [4]:
df

,ts,packageType,packageID,packageCh,messageType,messageTypeDesc,repeat,mmsi,dac,fid,...,swellPeriod,swellDirection,seaState,seaStateDesc,waterTemperature,precipitation,precipitationDesc,salinity,ice,iceDesc
0,2025-10-22 14:18:48.657,!ABVDM,6,A,8,Binary Broadcast Message,0,995331508,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,511.0,3,not available
1,2025-10-22 14:18:57.939,!ABVDM,9,A,8,Binary Broadcast Message,0,5631139,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
2,2025-10-22 14:19:04.256,!ABVDM,0,A,8,Binary Broadcast Message,0,5631129,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
3,2025-10-22 14:19:05.757,!ABVDM,2,A,8,Binary Broadcast Message,0,5631133,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
4,2025-10-22 14:19:56.641,!ABVDM,2,A,8,Binary Broadcast Message,0,995331514,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63207,2025-10-22 14:16:35.353,!ABVDM,5,B,8,Binary Broadcast Message,0,995331511,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
63208,2025-10-22 14:17:19.158,!ABVDM,9,A,8,Binary Broadcast Message,0,995331514,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
63209,2025-10-22 14:17:21.753,!ABVDM,4,A,8,Binary Broadcast Message,0,5631139,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available
63210,2025-10-22 14:17:27.565,!ABVDM,6,A,8,Binary Broadcast Message,0,5631129,1,31,...,63.0,360.0,13,Not available,501.0,7,not available or default,510.0,3,not available


In [9]:
duckdb.sql("INSTALL spatial")
duckdb.sql("LOAD spatial")

duckdb.sql(f'''
    SELECT distinct mmsi
    FROM df
    WHERE ST_Within(ST_Point(longitude, latitude), ST_GeomFromGeoJSON({outter_restricted_region})) 

''').fetchdf()

,mmsi
0,995331505


In [14]:
result = client.query(f'''    
    select *
    from pnav.ais_type24
    --from pnav.ptp_meteorological
    where mmsi = 995331505
    order by ts desc
    limit 10
''')

In [15]:
df = pd.DataFrame(result.result_rows)
df.columns = list(result.column_names)

df['ts'] = pd.to_datetime(df['ts'])  
df

,ts,packageType,packageID,packageCh,messageType,messageTypeDesc,repeat,mmsi,partNo,shipName,...,shipTypeDesc,vendor,model,serial,callsign,to_bow,to_stern,to_port,to_starboard,motherShip_mmsi
0,2025-10-28 12:24:22.870,!ABVDM,0,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
1,2025-10-14 18:51:49.528,!ABVDM,6,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
2,2025-08-27 15:18:37.152,!ABVDM,6,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
3,2025-05-21 00:16:57.851,!ABVDM,9,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
4,2025-04-13 17:10:37.341,!ABVDM,7,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
5,2025-04-13 08:37:32.636,!ABVDM,3,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
6,2025-02-21 23:24:28.497,!ABVDM,4,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
7,2025-02-21 10:52:08.268,!ABVDM,9,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
8,2025-02-21 01:28:22.267,!ABVDM,1,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
9,2025-02-12 07:30:39.437,!ABVDM,4,A,24,Static Data Report,0,995331505,0,TOMPOK UTARA WS,...,,,0,0,,0,0,0,0,0
